In [2]:
!git clone https://github.com/Cydia2018/ViT-cifar10-pruning.git

Cloning into 'ViT-cifar10-pruning'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 95 (delta 49), reused 83 (delta 40), pack-reused 0
Unpacking objects: 100% (95/95), done.


In [3]:
!git clone https://github.com/franklinnwren/ViT-pytorch.git

Cloning into 'ViT-pytorch'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 187 (delta 42), reused 27 (delta 27), pack-reused 130
Receiving objects: 100% (187/187), 21.31 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [4]:
%cd ViT-pytorch/

/content/ViT-pytorch


In [5]:
!pip install einops ml_collections

     |████████████████████████████████| 77 kB 5.8 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=5bf7e14d3f7958152b5e22f903cb6c3d67a04d0cb88d936248f7487007fb175e
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [10]:
!python train.py --name cifar10-100_500 --dataset cifar100 --model_type ViT-B_16 --train_batch_size 32 --pretrained_dir ViT-B_16.npz

流式输出内容被截断，只能显示最后 5000 行内容。
Validating... (loss=0.12898):  60% 94/157 [00:21<00:14,  4.36it/s]
Validating... (loss=0.33443):  60% 94/157 [00:22<00:14,  4.36it/s]
Validating... (loss=0.33443):  61% 95/157 [00:22<00:14,  4.37it/s]
Validating... (loss=0.22879):  61% 95/157 [00:22<00:14,  4.37it/s]
Validating... (loss=0.22879):  61% 96/157 [00:22<00:13,  4.36it/s]
Validating... (loss=0.20498):  61% 96/157 [00:22<00:13,  4.36it/s]
Validating... (loss=0.20498):  62% 97/157 [00:22<00:13,  4.34it/s]
Validating... (loss=0.20553):  62% 97/157 [00:22<00:13,  4.34it/s]
Validating... (loss=0.20553):  62% 98/157 [00:22<00:13,  4.34it/s]
Validating... (loss=0.13998):  62% 98/157 [00:23<00:13,  4.34it/s]
Validating... (loss=0.13998):  63% 99/157 [00:23<00:13,  4.35it/s]
Validating... (loss=0.21620):  63% 99/157 [00:23<00:13,  4.35it/s]
Validating... (loss=0.21620):  64% 100/157 [00:23<00:13,  4.35it/s]
Validating... (loss=0.12913):  64% 100/157 [00:23<00:13,  4.35it/s]
Validating... (loss=0.12913):  64

time for training: 1:57:44

In [36]:
from __future__ import absolute_import, division, print_function

import logging
import argparse
import os
import random
import numpy as np

from datetime import timedelta

import torch
import torch.distributed as dist

from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from models.modeling import VisionTransformer, CONFIGS
from utils.scheduler import WarmupLinearSchedule, WarmupCosineSchedule
from utils.data_utils import get_loader
from utils.dist_util import get_world_size

parser = argparse.ArgumentParser()
# Required parameters
parser.add_argument("--name", default="cifar100",
                    help="Name of this run. Used for monitoring.")
parser.add_argument("--dataset", choices=["cifar10", "cifar100"], default="cifar100",
                    help="Which downstream task.")
parser.add_argument("--model_type", choices=["ViT-B_16", "ViT-B_32", "ViT-L_16",
                                              "ViT-L_32", "ViT-H_14", "R50-ViT-B_16"],
                    default="ViT-B_16",
                    help="Which variant to use.")
parser.add_argument("--pretrained_dir", type=str, default="ViT-B_16.npz",
                    help="Where to search for pretrained ViT models.")
parser.add_argument("--pretrained_model", type=str, default="output/cifar10-100_500_checkpoint.bin")
parser.add_argument("--output_dir", default="output", type=str,
                    help="The output directory where checkpoints will be written.")

parser.add_argument("--img_size", default=224, type=int,
                    help="Resolution size")
parser.add_argument("--train_batch_size", default=512, type=int,
                    help="Total batch size for training.")
parser.add_argument("--eval_batch_size", default=64, type=int,
                    help="Total batch size for eval.")
parser.add_argument("--eval_every", default=100, type=int,
                    help="Run prediction on validation set every so many steps."
                          "Will always run one evaluation at the end of training.")

parser.add_argument("--learning_rate", default=3e-2, type=float,
                    help="The initial learning rate for SGD.")
parser.add_argument("--weight_decay", default=0, type=float,
                    help="Weight deay if we apply some.")
parser.add_argument("--num_steps", default=10000, type=int,
                    help="Total number of training epochs to perform.")
parser.add_argument("--decay_type", choices=["cosine", "linear"], default="cosine",
                    help="How to decay the learning rate.")
parser.add_argument("--warmup_steps", default=500, type=int,
                    help="Step of training to perform learning rate warmup for.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                    help="Max gradient norm.")

parser.add_argument("--local_rank", type=int, default=-1,
                    help="local_rank for distributed training on gpus")
parser.add_argument('--seed', type=int, default=42,
                    help="random seed for initialization")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                    help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument('--fp16', action='store_true',
                    help="Whether to use 16-bit float precision instead of 32-bit")
parser.add_argument('--fp16_opt_level', type=str, default='O2',
                    help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                          "See details at https://nvidia.github.io/apex/amp.html")
parser.add_argument('--loss_scale', type=float, default=0,
                    help="Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True.\n"
                          "0 (default value): dynamic loss scaling.\n"
                          "Positive power of 2: static loss scaling value.\n")
args = parser.parse_args("")

# Setup CUDA, GPU & distributed training
if args.local_rank == -1:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend='nccl',
                                          timeout=timedelta(minutes=60))
    args.n_gpu = 1

args.name = 'cifar100'
args.device = device

config = CONFIGS[args.model_type]
num_classes = 100
model = VisionTransformer(config, args.img_size, zero_head=True, num_classes=num_classes)
model.load_state_dict(torch.load(args.pretrained_model))
model.to(args.device)
print(args)
train_loader, test_loader = get_loader(args)

Namespace(dataset='cifar100', decay_type='cosine', device=device(type='cuda'), eval_batch_size=64, eval_every=100, fp16=False, fp16_opt_level='O2', gradient_accumulation_steps=1, img_size=224, learning_rate=0.03, local_rank=-1, loss_scale=0, max_grad_norm=1.0, model_type='ViT-B_16', n_gpu=1, name='cifar100', num_steps=10000, output_dir='output', pretrained_dir='ViT-B_16.npz', pretrained_model='output/cifar10-100_500_checkpoint.bin', seed=42, train_batch_size=512, warmup_steps=500, weight_decay=0)
Files already downloaded and verified
Files already downloaded and verified


In [37]:
import torch
import torch.nn.functional as F
from einops import rearrange
from torch import nn

total = 0
for name, m in model.named_modules():
    if 'select' in name:
        print(m)
        total += m.indexes.data.shape[0]

print("pause")

bn = torch.zeros(total)
index = 0
for name, m in model.named_modules():
    if 'select' in name:
        print(m)
        size = m.indexes.data.shape[0]
        bn[index:(index+size)] = m.indexes.data.abs().clone()
        index += size

channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
pause
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()
channel_selection()


In [38]:
percent = 0.1
y, i = torch.sort(bn)
thre_index = int(total * percent)
thre = y[thre_index]

# print(thre)

pruned = 0
cfg = []
cfg_mask = []
for k, (name, m) in enumerate(model.named_modules()):
    if 'select' in name:
        # print(k)
        # print(m)
        if k not in [14,31,48,65,82,99,116,133,150,167,184,201]:
            weight_copy = m.indexes.data.abs().clone()
            mask = weight_copy.gt(thre).float().cuda()
            thre_ = thre.clone()
            while (torch.sum(mask) % 8 != 0): # heads
                thre_ = thre_ - 0.001
                mask = weight_copy.gt(thre_).float().cuda()
        else:
            weight_copy = m.indexes.data.abs().clone()
            mask = weight_copy.gt(thre).float().cuda()
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        m.indexes.data.mul_(mask)
        cfg.append(int(torch.sum(mask)))
        cfg_mask.append(mask.clone())
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))

layer index: 14 	 total channel: 3072 	 remaining channel: 2726
layer index: 19 	 total channel: 768 	 remaining channel: 416
layer index: 31 	 total channel: 3072 	 remaining channel: 2806
layer index: 36 	 total channel: 768 	 remaining channel: 424
layer index: 48 	 total channel: 3072 	 remaining channel: 2890
layer index: 53 	 total channel: 768 	 remaining channel: 664
layer index: 65 	 total channel: 3072 	 remaining channel: 2942
layer index: 70 	 total channel: 768 	 remaining channel: 632
layer index: 82 	 total channel: 3072 	 remaining channel: 2964
layer index: 87 	 total channel: 768 	 remaining channel: 520
layer index: 99 	 total channel: 3072 	 remaining channel: 2983
layer index: 104 	 total channel: 768 	 remaining channel: 768
layer index: 116 	 total channel: 3072 	 remaining channel: 2968
layer index: 121 	 total channel: 768 	 remaining channel: 712
layer index: 133 	 total channel: 3072 	 remaining channel: 2982
layer index: 138 	 total channel: 768 	 remaining 

In [39]:
pruned_ratio = pruned / total
print('Pre-processing Successful with pruned ratio', pruned_ratio.item())
print(cfg)

Pre-processing Successful with pruned ratio 0.07523871958255768
[2726, 416, 2806, 424, 2890, 664, 2942, 632, 2964, 520, 2983, 768, 2968, 712, 2982, 616, 2973, 640, 2945, 728, 2970, 640, 3024, 680]


In [40]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def test(model, test_loader):
  model.eval()
  all_preds, all_label = [], []
  epoch_iterator = tqdm(test_loader,
                        desc="Validating... (loss=X.X)",
                        bar_format="{l_bar}{r_bar}",
                        dynamic_ncols=True,
                        disable=args.local_rank not in [-1, 0])
  loss_fct = torch.nn.CrossEntropyLoss()
  for step, batch in enumerate(epoch_iterator):
      batch = tuple(t.to(args.device) for t in batch)
      x, y = batch
      with torch.no_grad():
          logits = model(x)[0]

          eval_loss = loss_fct(logits, y)
          # eval_losses.update(eval_loss.item())

          preds = torch.argmax(logits, dim=-1)

      if len(all_preds) == 0:
          all_preds.append(preds.detach().cpu().numpy())
          all_label.append(y.detach().cpu().numpy())
      else:
          all_preds[0] = np.append(
              all_preds[0], preds.detach().cpu().numpy(), axis=0
          )
          all_label[0] = np.append(
              all_label[0], y.detach().cpu().numpy(), axis=0
          )
      # epoch_iterator.set_description("Validating... (loss=%2.5f)" % eval_losses.val)

  all_preds, all_label = all_preds[0], all_label[0]
  accuracy = simple_accuracy(all_preds, all_label)
  print("Valid Accuracy: %2.5f" % accuracy)

In [41]:
test(model, test_loader)

Validating... (loss=X.X): 100%|| 157/157 [00:36<00:00,  4.35it/s]

Valid Accuracy: 0.74460


In [ ]:
#if running on a local divice, comment these lines
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
